In [2]:
# ========================
# 0. IMPORTS
# ========================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, precision_score, recall_score, f1_score

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, Dense, LSTM, RepeatVector, TimeDistributed, 
                                     MultiHeadAttention, LayerNormalization, Add, 
                                     Conv1D, GlobalAveragePooling1D)

from tensorflow.keras.callbacks import EarlyStopping
from einops import rearrange

import os

In [3]:
# ========================
# 1. CONFIGURATION
# ========================
INPUT_STEPS = 12
FORECAST_STEPS = 12
TEST_RATIO = 0.3

# Tuning parameters
EPOCHS_LIST = [10, 20]
BATCH_SIZES = [64, 128]

# Simulation parameters
WINDOW_SIZE_SIMULATION = 12  # 6 hours → 6×6=36 steps (if 10-min data)
THRESHOLD_PERCENTILE = 95

SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [4]:
# ========================
# 2. DEVICE SETUP
# ========================
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        tf.config.set_visible_devices(gpus[0], 'GPU')
        print("✅ GPU is available and will be used.")
    except RuntimeError as e:
        print(e)
else:
    print("⚠️ No GPU detected, running on CPU.")

✅ GPU is available and will be used.


In [5]:
# ========================
# 3. LOAD AND PREPROCESS DATA
# ========================
file_path = '../rfcc_longest_active_window.csv'
df = pd.read_csv(file_path)
df['DateTime'] = pd.to_datetime(df['DateTime'], errors='coerce')
df.set_index('DateTime', inplace=True)

# Clean and normalize
df.dropna(axis=1, thresh=int(0.7 * len(df)), inplace=True)
df.ffill(inplace=True)
df.bfill(inplace=True)

scaler = MinMaxScaler()
scaled = scaler.fit_transform(df.values)
df_scaled = pd.DataFrame(scaled, index=df.index, columns=df.columns).astype(np.float32)

print(f"✅ Scaled dataset shape: {df_scaled.shape}")

✅ Scaled dataset shape: (62174, 26)


In [6]:
# ========================
# 4. SEQUENTIAL TRAIN/TEST SPLIT
# ========================
split_idx = int((1 - TEST_RATIO) * len(df_scaled))
train_data = df_scaled.iloc[:split_idx]
test_data = df_scaled.iloc[split_idx:]

print(f"✅ Training samples: {len(train_data)}, Testing samples: {len(test_data)}")

# ========================
# 5. PATCH SEQUENCE GENERATOR
# ========================
def create_patch_sequences(data, input_steps, forecast_steps):
    X, y = [], []
    for i in range(len(data) - input_steps - forecast_steps):
        X.append(data[i:i+input_steps])
        y.append(data[i+input_steps:i+input_steps+forecast_steps])
    return np.array(X, dtype=np.float32), np.array(y, dtype=np.float32)

X_train, y_train = create_patch_sequences(train_data.values, INPUT_STEPS, FORECAST_STEPS)
X_test, y_test = create_patch_sequences(test_data.values, INPUT_STEPS, FORECAST_STEPS)

print(f"✅ Training sequences: {X_train.shape}, Testing sequences: {X_test.shape}")

✅ Training samples: 43521, Testing samples: 18653
✅ Training sequences: (43497, 12, 26), Testing sequences: (18629, 12, 26)


In [7]:
# ========================
# 6. BUILD PATCHTST MODEL
# ========================
def build_patchtst(input_steps, num_features, patch_len=3, embed_dim=128, num_heads=4):
    assert input_steps % patch_len == 0, "Input steps must be divisible by patch length."
    num_patches = input_steps // patch_len

    inp = Input(shape=(input_steps, num_features))
    x = rearrange(inp, 'b (p l) f -> b p (l f)', p=num_patches, l=patch_len)
    x = Dense(embed_dim)(x)

    attn = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)(x, x)
    x = Add()([x, attn])
    x = LayerNormalization()(x)

    x = GlobalAveragePooling1D()(x)
    x = RepeatVector(FORECAST_STEPS)(x)

    x = LSTM(embed_dim, return_sequences=True)(x)
    out = TimeDistributed(Dense(num_features))(x)

    model = Model(inp, out)
    model.compile(optimizer='adam', loss='mse')
    return model

In [9]:
# ========================
# 7. TRAINING + TUNING
# ========================
best_val_mse = np.inf
best_model = None
history_records = []

for epochs in EPOCHS_LIST:
    for batch_size in BATCH_SIZES:
        print(f"\n🔵 Training model with epochs={epochs}, batch_size={batch_size}")
        
        model = build_patchtst(INPUT_STEPS, X_train.shape[2])
        es = EarlyStopping(patience=5, restore_best_weights=True)

        history = model.fit(X_train, y_train,
                            validation_split=0.1,
                            epochs=epochs,
                            batch_size=batch_size,
                            callbacks=[es],
                            verbose=1,
                            shuffle=False)
        
        val_preds = model.predict(X_test, batch_size=batch_size)
        val_mse = mean_squared_error(y_test.reshape(-1), val_preds.reshape(-1))
        val_mae = mean_absolute_error(y_test.reshape(-1), val_preds.reshape(-1))

        print(f"✅ Validation RMSE: {val_mse:.5f}, MAE: {val_mae:.5f}")

        history_records.append({
            "epochs": epochs,
            "batch_size": batch_size,
            "val_mse": val_mse,
            "val_mae": val_mae
        })

        if val_mse < best_val_mse:
            best_val_mse = val_mse
            best_model = model

# Save history
history_df = pd.DataFrame(history_records)
history_df.to_csv("patchtst_tuning_history.csv", index=False)
print("\n📋 Tuning Results Summary:")
print(history_df)

# Save best model
best_model.save("best_patchtst_forecaster.h5")
print("\n✅ Best PatchTST model saved.")


🔵 Training model with epochs=10, batch_size=64
Epoch 1/10
612/612 [==============================] - 7s 8ms/step - loss: 0.0067 - val_loss: 0.0034
Epoch 2/10
612/612 [==============================] - 8s 13ms/step - loss: 0.0047 - val_loss: 0.0033
Epoch 3/10
612/612 [==============================] - 10s 16ms/step - loss: 0.0049 - val_loss: 0.0036
Epoch 4/10
612/612 [==============================] - 9s 15ms/step - loss: 0.0051 - val_loss: 0.0043
Epoch 5/10
612/612 [==============================] - 10s 16ms/step - loss: 0.0053 - val_loss: 0.0043
Epoch 6/10
612/612 [==============================] - 10s 16ms/step - loss: 0.0054 - val_loss: 0.0040
Epoch 7/10
292/292 [==============================] - 3s 7ms/step
✅ Validation RMSE: 0.04206, MAE: 0.12982

🔵 Training model with epochs=10, batch_size=128
Epoch 1/10
306/306 [==============================] - 8s 16ms/step - loss: 0.0102 - val_loss: 0.0050
Epoch 2/10
306/306 [==============================] - 4s 13ms/step - loss: 0.0066 - val